In [11]:
import sklearn.datasets as skdata
import torch as t
import numpy as np
from collections import OrderedDict
from dataclasses import dataclass
from sklearn.metrics import accuracy_score

In [12]:
class LogReg(t.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = t.nn.Linear(20, 8)
        self.fc2 = t.nn.Linear(8, 1)
    
    def forward(self, batch_x):
        x = t.nn.functional.relu(self.fc1(batch_x))
        batch_y_hat = t.sigmoid(self.fc2(x))
        return t.squeeze(batch_y_hat, dim=1)

In [13]:
@dataclass
class Hyperparams:
    n_epochs: int
    batch_size: int
    lr: float
    true_cutoff: float

In [14]:
def gen_datasets(n_samples):
    X, y = skdata.make_classification(
        n_samples=n_samples,
        n_features=20,
        n_informative=10,
        n_redundant=7,
        n_repeated=3,
        n_classes=2,
        flip_y=0.05,  # larger values make the task hard
        class_sep=0.5,  # larger values makes the task easy
        random_state=10
    )
    
    train_size = int(n_samples * 0.7)
    val_size = int(n_samples * 0.2)

    train_X = X[:train_size]
    train_y = y[:train_size]
    trainset = t.utils.data.TensorDataset(
        t.from_numpy(train_X).to(t.float32),
        t.from_numpy(train_y).to(t.float32)
    )

    val_X = X[train_size : train_size + val_size]
    val_y = y[train_size : train_size + val_size]
    valset = t.utils.data.TensorDataset(
        t.from_numpy(val_X).to(t.float32),
        t.from_numpy(val_y).to(t.float32)
    )

    test_X = X[train_size + val_size :]
    test_y = y[train_size + val_size :]
    testset = t.utils.data.TensorDataset(
        t.from_numpy(test_X).to(t.float32),
        t.from_numpy(test_y).to(t.float32)
    )
    
    return trainset, valset, testset

In [15]:
hparams = Hyperparams(n_epochs=10, batch_size=32, lr=0.005, true_cutoff=0.5)

In [16]:
trainset, valset, testset = gen_datasets(n_samples=100_000)
traindl = t.utils.data.DataLoader(trainset, batch_size=hparams.batch_size)
valdl = t.utils.data.DataLoader(valset, batch_size=5000)

In [17]:
model = LogReg()
optim = t.optim.Adam(model.parameters(), lr=hparams.lr)
loss_fn = t.nn.BCELoss()

In [18]:
for epoch in range(1, hparams.n_epochs + 1):
    y_hat = t.Tensor([])
    y = t.Tensor([])
    batch_losses = []
    model.train()
    with t.enable_grad():
        for batch_X, batch_y in traindl:
            optim.zero_grad()
            batch_y_hat = model.forward(batch_X)
            loss = loss_fn(batch_y_hat, batch_y)
            loss.backward()
            optim.step()
            batch_losses.append(loss.detach().item())
            y_hat = t.cat((y_hat, batch_y_hat.detach()))
            y = t.cat((y, batch_y.detach()))
    y_pred = y_hat > hparams.true_cutoff
    train_acc = accuracy_score(y, y_pred)
    train_loss = sum(batch_losses)/len(batch_losses)
    
    y_hat = t.Tensor([])
    y = t.Tensor([])
    batch_losses = []
    model.eval()
    with t.no_grad():
        for batch_X, batch_y in valdl:
            batch_y_hat = model(batch_X)
            loss = loss_fn(batch_y_hat, batch_y)
            batch_losses.append(loss.detach().item())
            y_hat = t.cat((y_hat, batch_y_hat))
            y = t.cat((y, batch_y))
    y_pred = y_hat > hparams.true_cutoff
    val_acc = accuracy_score(y, y_pred)
    val_loss = sum(batch_losses)/len(batch_losses)
    
    print(f"Epoch {epoch}: Train accuracy={train_acc:.3f}, Val accuracy={val_acc:.3f}, Train loss={train_loss:.3f}, Val loss={val_loss:.3f}")

Epoch 1: Train accuracy=0.818, Val accuracy=0.849, Train loss=0.410, Val loss=0.362
Epoch 2: Train accuracy=0.855, Val accuracy=0.869, Train loss=0.353, Val loss=0.337
Epoch 3: Train accuracy=0.868, Val accuracy=0.873, Train loss=0.333, Val loss=0.327
Epoch 4: Train accuracy=0.874, Val accuracy=0.874, Train loss=0.325, Val loss=0.320
Epoch 5: Train accuracy=0.878, Val accuracy=0.876, Train loss=0.319, Val loss=0.315
Epoch 6: Train accuracy=0.881, Val accuracy=0.878, Train loss=0.315, Val loss=0.313
Epoch 7: Train accuracy=0.883, Val accuracy=0.880, Train loss=0.312, Val loss=0.314
Epoch 8: Train accuracy=0.886, Val accuracy=0.883, Train loss=0.310, Val loss=0.311
Epoch 9: Train accuracy=0.888, Val accuracy=0.884, Train loss=0.308, Val loss=0.309
Epoch 10: Train accuracy=0.893, Val accuracy=0.888, Train loss=0.302, Val loss=0.309


In [10]:
testdl = t.utils.data.DataLoader(testset, batch_size=5000)
y_hat = t.Tensor([])
y = t.Tensor([])
batch_losses = []
model.eval()
with t.no_grad():
    for batch_X, batch_y in testdl:
        batch_y_hat = model(batch_X)
        loss = loss_fn(batch_y_hat, batch_y)
        batch_losses.append(loss.detach().item())
        y_hat = t.cat((y_hat, batch_y_hat))
        y = t.cat((y, batch_y))
y_pred = y_hat > hparams.true_cutoff
test_acc = accuracy_score(y, y_pred)
test_loss = sum(batch_losses)/len(batch_losses)
print(f"Test accuracy={test_acc:.3f} Test loss={test_loss:.3f}")

Test accuracy=0.874 Test loss=0.328


In [21]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1933,  0.3040,  0.2339, -0.4712, -2.0335, -0.1256,  0.3059, -1.0716,
           0.2664, -0.2187,  1.4983,  1.5031, -0.9345, -1.2143,  0.6086,  0.4412,
          -0.7629, -0.4040,  0.9199,  0.5362],
         [-0.8095,  0.3273, -0.1031,  0.5970, -0.2447,  0.0505,  0.8312, -0.9991,
           0.1177,  0.4486,  1.0565,  0.9103,  0.8362, -1.6116,  0.7527, -0.4848,
          -0.3421,  1.4960, -0.3365, -0.0799],
         [-1.6605,  1.0875,  0.1025, -0.5268,  0.7196,  0.2690, -0.7103, -0.1966,
           1.3231,  0.8041, -0.1564,  0.0364, -0.2973,  0.6151, -0.5530,  0.1251,
          -0.2612,  0.7585,  0.1787, -0.4786],
         [-1.2165, -0.3393, -0.0241, -0.8325, -0.3739, -0.3921,  0.9219,  0.4433,
          -0.0687,  0.4376, -0.6073, -0.8516,  0.7861,  0.6547,  0.6940,  0.0043,
          -0.2669,  0.3149, -0.7328, -0.4379],
         [ 0.7825, -0.3184, -0.0480, -0.1963, -2.3431,  0.3283,  0.8623, -0.8529,
          -0.1996, -0.4762,  1.2222,  1.2832, -0.42